# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,0.85,55,20,5.14,CA,1715117070
1,1,gizo,-8.1030,156.8419,28.26,83,89,6.82,SB,1715117070
2,2,port augusta,-32.5000,137.7667,14.34,60,83,2.84,AU,1715117070
3,3,asadabad,34.7824,48.1185,10.97,77,24,3.88,IR,1715117070
4,4,arteche,12.2645,125.4048,26.22,87,100,2.84,PH,1715117070


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
city_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    size = "Humidity"
)

# Display the map
city_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
perferred_city_df = city_data_df[ 
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Max Temp"] < 27) &
    # (city_data_df["Humidity"] < 75) &
    (city_data_df["Cloudiness"] == 0) &
    (city_data_df["Wind Speed"] < 4.5)
]

# Drop any rows with null values
perferred_city_df=perferred_city_df.dropna()

# Display sample data
perferred_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,bethel,41.3712,-73.4140,26.68,29,0,4.12,US,1715117071
40,40,olinda,-8.0089,-34.8553,26.85,83,0,2.57,BR,1715116883
61,61,port saint john's,-31.6229,29.5448,23.78,49,0,2.87,ZA,1715117077
84,84,gebeit,21.0667,36.3167,22.85,44,0,2.33,SD,1715117080
140,140,broome,42.2506,-75.8330,24.21,42,0,3.13,US,1715117086
144,144,al kharijah,25.4514,30.5464,22.49,23,0,3.99,EG,1715117087
249,249,karratha,-20.7377,116.8463,24.25,69,0,1.78,AU,1715117099
250,250,toliara,-23.3500,43.6667,22.20,65,0,3.91,MG,1715117099
253,253,south boston,42.3334,-71.0495,26.92,40,0,3.60,US,1715117071
317,317,nova vicosa,-17.8919,-39.3719,25.54,82,0,2.82,BR,1715117108


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perferred_city_df[["City", 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,bethel,US,41.3712,-73.4140,29,
40,olinda,BR,-8.0089,-34.8553,83,
61,port saint john's,ZA,-31.6229,29.5448,49,
84,gebeit,SD,21.0667,36.3167,44,
140,broome,US,42.2506,-75.8330,42,
144,al kharijah,EG,25.4514,30.5464,23,
249,karratha,AU,-20.7377,116.8463,69,
250,toliara,MG,-23.3500,43.6667,65,
253,south boston,US,42.3334,-71.0495,40,
317,nova vicosa,BR,-17.8919,-39.3719,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {
        "apiKey": geoapify_key,
        "categories":"accommodation.hotel"
        }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Latitude= row["Lat"]
    Longitude= row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
olinda - nearest hotel: Fidji Motel
port saint john's - nearest hotel: Outback Inn
gebeit - nearest hotel: No hotel found
broome - nearest hotel: No hotel found
al kharijah - nearest hotel: Kharga Hotel
karratha - nearest hotel: Karratha International Hotel
toliara - nearest hotel: Ambary
south boston - nearest hotel: Quarters on Dots
nova vicosa - nearest hotel: Pousada Pontal da Barra
inhambane - nearest hotel: Hotel de Inhambane
mahajanga - nearest hotel: Vahiny Hotel
vila velha - nearest hotel: Hotel Prainha
holualoa - nearest hotel: Kona Hotel
passo fundo - nearest hotel: Turis Hotel Passo Fundo
esmeraldas - nearest hotel: No hotel found
arinos - nearest hotel: No hotel found
khash - nearest hotel: No hotel found
indramayu - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
10,bethel,US,41.3712,-73.4140,29,Hampton Inn Danbury
40,olinda,BR,-8.0089,-34.8553,83,Fidji Motel
61,port saint john's,ZA,-31.6229,29.5448,49,Outback Inn
84,gebeit,SD,21.0667,36.3167,44,No hotel found
140,broome,US,42.2506,-75.8330,42,No hotel found
144,al kharijah,EG,25.4514,30.5464,23,Kharga Hotel
249,karratha,AU,-20.7377,116.8463,69,Karratha International Hotel
250,toliara,MG,-23.3500,43.6667,65,Ambary
253,south boston,US,42.3334,-71.0495,40,Quarters on Dots
317,nova vicosa,BR,-17.8919,-39.3719,82,Pousada Pontal da Barra


In [23]:
# editing out the rows without hotels to remove clutter from the map. 
# Create a new DataFrame excluding rows with "No hotel found"
filtered_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]

# Check the new DataFrame
print(filtered_hotel_df)

                  City Country      Lat       Lng  Humidity  \
10              bethel      US  41.3712  -73.4140        29   
40              olinda      BR  -8.0089  -34.8553        83   
61   port saint john's      ZA -31.6229   29.5448        49   
144        al kharijah      EG  25.4514   30.5464        23   
249           karratha      AU -20.7377  116.8463        69   
250            toliara      MG -23.3500   43.6667        65   
253       south boston      US  42.3334  -71.0495        40   
317        nova vicosa      BR -17.8919  -39.3719        82   
336          inhambane      MZ -23.8650   35.3833        74   
339          mahajanga      MG -15.7167   46.3167        64   
345         vila velha      BR -20.3297  -40.2925        78   
352           holualoa      US  19.6228 -155.9522        76   
388        passo fundo      BR -28.2628  -52.4067        60   

                       Hotel Name  
10            Hampton Inn Danbury  
40                    Fidji Motel  
61       

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

# Configure the map plot
hotel_plot_1 = filtered_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]    
)

# Display the map
hotel_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)